In [1]:
import os 
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from pathlib import Path
import os
import importlib
import warnings
import gc

pd.set_option('mode.chained_assignment',  None) 
warnings.simplefilter(action='ignore', category=FutureWarning) 

module_path='model'
if module_path not in sys.path:
    sys.path.append(module_path)
    
import generation
from generation import *

In [3]:
# data = generate_adm()

# proc = generate_proc(data)

# out = generate_out(data)

# chart = generate_chart(data)

# labs = generate_labs(data)

# microlabs = generate_microlabs(data)

# meds = generate_meds(data)

# ing = generate_ing(data)

100%|██████████| 18302/18302 [00:25<00:00, 706.74it/s]


[Complete generate microlab events]
[Complete generate medication events]
[Complete generate ingredient events]


In [4]:
# data.to_csv('check_point_data.csv.gz', compression = 'gzip')
# proc.to_csv('check_point_proc.csv.gz', compression = 'gzip')
# out.to_csv('check_point_out.csv.gz', compression = 'gzip')
# chart.to_csv('check_point_chart.csv.gz', compression = 'gzip')
# labs.to_csv('check_point_labs.csv.gz', compression = 'gzip')
# microlabs.to_csv('check_point_microlabs.csv.gz', compression = 'gzip')
# meds.to_csv('check_point_meds.csv.gz', compression = 'gzip')
# ing.to_csv('check_point_ing.csv.gz', compression = 'gzip')

In [2]:
# loading check point

import pandas as pd
import numpy as np
from tqdm import tqdm

data = pd.read_csv('check_point_data.csv.gz', index_col = 0, compression = 'gzip')
ing = pd.read_csv('check_point_ing.csv.gz', index_col = 0, compression = 'gzip')
chart = pd.read_csv('check_point_chart.csv.gz', index_col = 0, compression = 'gzip')
labs = pd.read_csv('check_point_labs.csv.gz', index_col = 0, compression = 'gzip')
proc = pd.read_csv('check_point_proc.csv.gz', index_col = 0, compression = 'gzip')
out = pd.read_csv('check_point_out.csv.gz', index_col = 0, compression = 'gzip')
meds = pd.read_csv('check_point_meds.csv.gz', index_col = 0, compression = 'gzip')
microlabs = pd.read_csv('check_point_microlabs.csv.gz', index_col = 0, compression = 'gzip')

In [3]:
include_start_time = 1*24
include_end_time =60*24

def cohort_restirction(include_start_time,include_end_time,data, ing, chart, labs, proc, out, meds, microlabs):
    print("include start time",include_start_time)
    print("include end time",include_end_time)
    
    data=data[(data['los'] >= include_start_time)]
    data=data[(data['los'] <= include_end_time)]
    hids=data['stay_id'].unique()
    print('num of patient: ', len(data.subject_id.unique()))
    print('num of stay: ', len(hids))

    # cond=cond[cond['stay_id'].isin(data['stay_id'])]
    
    ###MEDS

    meds=meds[meds['stay_id'].isin(data['stay_id'])]
    meds=meds[meds['start_time'] <= include_end_time]
    meds.loc[meds.stop_time > include_end_time, 'stop_time']=include_end_time
        
    ###ING

    ing=ing[ing['stay_id'].isin(data['stay_id'])]
    ing=ing[ing['start_time'] <= include_end_time]
    ing.loc[ing.stop_time > include_end_time, 'stop_time']=include_end_time
                
    ###PROCS

    proc=proc[proc['stay_id'].isin(data['stay_id'])]
    proc=proc[proc['start_time']<=include_end_time]
    proc.loc[proc.stop_time > include_end_time, 'stop_time']=include_end_time
    
    ###microlabs

    microlabs=microlabs[microlabs['stay_id'].isin(data['stay_id'])]
    microlabs=microlabs[microlabs['start_time']<=include_end_time]
        
    ###OUT

    out=out[out['stay_id'].isin(data['stay_id'])]
    out=out[out['start_time']<=include_end_time]
        
    ###CHART

    chart=chart[chart['stay_id'].isin(data['stay_id'])]
    chart=chart[chart['start_time']<=include_end_time]
        
    ###LAB

    labs=labs[labs['stay_id'].isin(data['stay_id'])]
    labs=labs[labs['start_time']<=include_end_time]
    
    return  data, ing, chart, labs, proc, out, meds, microlabs

data_new, ing_new, chart_new, labs_new, proc_new, out_new, meds_new, microlabs_new = cohort_restirction(include_start_time,include_end_time ,data, ing, chart, labs, proc, out, meds, microlabs)

include start time 24
include end time 1440
num of patient:  42253
num of stay:  57699


In [4]:
final_meds=pd.DataFrame()
final_ing=pd.DataFrame()
final_proc=pd.DataFrame()
final_out=pd.DataFrame()
final_chart=pd.DataFrame()
final_labs=pd.DataFrame()
final_micro=pd.DataFrame()

meds_new=meds_new.sort_values(by=['start_time'])
ing_new=ing_new.sort_values(by=['start_time'])
proc_new=proc_new.sort_values(by=['start_time'])
microlabs_new=microlabs_new.sort_values(by=['start_time'])
out_new=out_new.sort_values(by=['start_time'])
chart_new=chart_new.sort_values(by=['start_time'])
labs_new=labs_new.sort_values(by=['start_time'])

hids=data_new['stay_id'].unique()

In [5]:
sample_data = pd.concat([chart_new[['stay_id', 'itemid']], labs_new[['stay_id', 'itemid']]], axis = 0)

In [6]:
# Specify the item_ids we are interested in
required_item_ids = {'HR', 'ABPs', 'ABPd', 'Lactate'}

# Find the stay_ids that have all the required item_ids at least once
valid_stay_ids = sample_data[sample_data['itemid'].isin(required_item_ids)].groupby('stay_id')['itemid'].nunique()
valid_stay_ids = valid_stay_ids[valid_stay_ids == len(required_item_ids)].index

In [7]:
meds_1 = meds_new[meds_new['stay_id'].isin(valid_stay_ids)]
ing_1 = ing_new[ing_new['stay_id'].isin(valid_stay_ids)]
proc_1 = proc_new[proc_new['stay_id'].isin(valid_stay_ids)]
out_1 = out_new[out_new['stay_id'].isin(valid_stay_ids)]
chart_1 = chart_new[chart_new['stay_id'].isin(valid_stay_ids)]
labs_1 = labs_new[labs_new['stay_id'].isin(valid_stay_ids)]
data_1 = data_new[data_new['stay_id'].isin(valid_stay_ids)]
microlabs_1 = microlabs_new[microlabs_new['stay_id'].isin(valid_stay_ids)]

In [8]:
#예상 관측치 수
data_1['los'].sum()

3007634

In [9]:
#총 STAY 수
len(valid_stay_ids)

20809

In [10]:
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment',  None) 

local = '/Users/DAHS/Desktop/MIMICIV2.2_PREPROC/data'

final_meds = meds_1.copy()
final_ing= ing_1.copy()
final_proc= proc_1.copy() 
final_out= out_1.copy()
final_chart= chart_1.copy()
final_labs= labs_1.copy()
final_micro = microlabs_1.copy()

feat_med = True
feat_ing = True
feat_proc = True
feat_out = True
feat_chart = True
impute = True
feat_lab = True
feat_micro = True

tabularization(feat_med, feat_ing, feat_out, feat_chart, feat_lab, feat_micro,feat_proc,
                   final_meds, final_ing, final_proc, final_out, final_chart, final_labs, final_micro,
                   valid_stay_ids, data_1)

data_1.to_csv(local+'/demo.csv')

# Unique gender:  2
# Unique ethnicity:  33
Number of patient:  18685
Number of stay:  20809
Expected value of observation:  3007634



Tabularize EHR for total stay 20,809: 100%|██████████| 20809/20809 [19:32:40<00:00,  3.38s/it]  

[ SUCCESSFULLY SAVED TOTAL UNIT STAY DATA ]
